To-do:
* нормализация
* бинарные --> категориальные
* добавить расчет Kaggle score
* понять, почему первый submit на методе Ньютона (100 итераций?) дал рез-т выше, чем сошедшийся lbfgs
    * посчитать score на random, сравнить
* глобально: добавить решающее дерево, random forest, boosting
* визуализация результатов predict

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn.linear_model

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
M = train['Id'].count() # размер обучающей выборки

## 1. Посмотрим на данные

In [ ]:
train.head()

In [ ]:
train.describe()

Видно, что:
1. Evaluatuion надо усреднить
2. Горизонтальные расстояния положительные, вертикальное может быть меньше 0
3. Hillshade_Noon тоже можно усреднить

### 3. Hillshade

In [ ]:
train.pivot_table(index = 'Id', values = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']).corr()

Освещенность в 3 часа и в 9 хорошо коррелируют.
* собрать их в один признак?

## 2. Визуализация

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.figure()
plt.subplot(211)
plt.scatter(train['Horizontal_Distance_To_Hydrology'].values, \
         train['Vertical_Distance_To_Hydrology'].values, c = train['Cover_Type'], s = 2)
plt.title = 'train'

plt.subplot(212)
plt.plot(test['Horizontal_Distance_To_Hydrology'].values, \
         test['Vertical_Distance_To_Hydrology'].values, 'go', markersize = 2)
plt.title = 'test'

Карта расстояний до ближайшей воды

## 3. Отделение cross-validation set

In [ ]:
threshold = int(M * 0.6) # 60% обучающей выборки - новый train, 40% - cross-validation
train_new = train[:threshold]
cv = train[threshold:]
print (M)
print (train_new.shape)
print (cv.shape)

## 3.5 Нормализация признаков-расстояний

In [ ]:
train['Elevation'] = (train['Elevation'] - train['Elevation'].mean())/train['Elevation'].std()
train['Horizontal_Distance_To_Hydrology'] = (train['Horizontal_Distance_To_Hydrology'] - train['Horizontal_Distance_To_Hydrology'].mean())/train['Horizontal_Distance_To_Hydrology'].std()
train['Vertical_Distance_To_Hydrology'] = (train['Vertical_Distance_To_Hydrology'] - train['Vertical_Distance_To_Hydrology'].mean())/train['Vertical_Distance_To_Hydrology'].std()
train['Horizontal_Distance_To_Roadways'] = (train['Horizontal_Distance_To_Roadways'] - train['Horizontal_Distance_To_Roadways'].mean())/train['Horizontal_Distance_To_Roadways'].std()

In [ ]:
train['Aspect'] = (train['Aspect'] - train['Aspect'].mean())/train['Aspect'].std()
train['Slope'] = (train['Slope'] - train['Slope'].mean())/train['Slope'].std()
train['Hillshade_9am'] = (train['Hillshade_9am'] - train['Hillshade_9am'].mean())/train['Hillshade_9am'].std()
train['Hillshade_Noon'] = (train['Hillshade_Noon'] - train['Hillshade_Noon'].mean())/train['Hillshade_Noon'].std()
train['Hillshade_3pm'] = (train['Hillshade_3pm'] - train['Hillshade_3pm'].mean())/train['Hillshade_3pm'].std()

## 4. Логическая регрессия

In [ ]:
'''logistic_reg = sklearn.linear_model.LogisticRegression(random_state = 1, \
                                                  solver = 'lbfgs', \
                                                  tol = 1e-4, \
                                                  max_iter = 500, \
                                                  n_jobs = -1)
                                                    '''

* Логистическая регрессия по методу Ньютона с 500 (и 100) итерациями (tol = 1e-2, 1e-3) не сходится (line search не сходится).

score(train) = 0.666446208113
score(cv) = 0.5625

* Метод lbfgs - сошлась,
score (train) = 0.605709876543
score(cv) = 0.486937830688
Kaggle score на test: 0.05936

* sag max_iter = 500 - не сошлась, 
score(train) = 0.554453262787
score(cv) = 0.449900793651

In [ ]:
X = train_new.drop('Cover_Type', axis = 1)
y = train_new['Cover_Type'].values
#logistic_reg.fit(X, y)

In [ ]:
#pred = logistic_reg.predict(cv.drop('Cover_Type', axis = 1))

In [ ]:
#print (logistic_reg.score(X, y))
#print (logistic_reg.score(cv.drop('Cover_Type', axis = 1), cv['Cover_Type'].values))

Опционально: теперь обучимся на всей обучающей выборке, предскажем значения Cover Type для тестовой и сохраним резульатат в файл для последующего самбита 

In [ ]:
log_reg = sklearn.linear_model.LogisticRegression(random_state = 1, \
                                                  solver = 'lbfgs', \
                                                  tol = 1e-4, \
                                                  max_iter = 1500, \
                                                  n_jobs = -1)

In [ ]:
X = train.drop('Cover_Type', axis = 1)
y = train['Cover_Type'].values
log_reg.fit(X, y)
print (log_reg.score(X, y))
print (log_reg.score(cv.drop('Cover_Type', axis = 1), cv['Cover_Type'].values))

In [ ]:
pred = log_reg.predict(test)
pred_df = pd.DataFrame({'Cover_Type' : pred}, test['Id'])
pred_df.head()

In [ ]:
pred_df.to_csv('output.csv')